In [ ]:
import cv2
from PIL import Image, ImageOps
import argparse
import os
import glob
import matplotlib.pyplot as plt
import multiprocessing as mp
from joblib import Parallel, delayed
import pydicom
import time
import numpy as np
import torch
import random
import pandas as pd
from pydicom.pixel_data_handlers import apply_windowing
import imageio
import time
from tqdm.notebook import tqdm

import shutil

In [ ]:
meta = pd.read_csv('mass_case_description_train_set.csv')

In [ ]:
meta['id'] = meta['image file path'].apply(lambda x: x.split('/')[0])

In [ ]:
train.to_csv('train_aug_cibs_ddsm1.csv')

### Convert image to png and storage

In [ ]:
def get_all_files_in_folder(folder_path):
    # print(folder_path)
    all_files = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            all_files.append(file_path)
    return all_files

In [ ]:
get_all_files_in_folder('Train-Mass-Roi/manifest-LyDgOQGl3853937313152078328/CBIS-DDSM/Mass-Training_P_00001_LEFT_CC/')
get_all_files_in_folder('Train-Mass-Roi\manifest-LyDgOQGl3853937313152078328\CBIS-DDSM\Mass-Training_P_00001_LEFT_CC_1')

In [ ]:
# kiểm tra coi folder nào có 4 ảnh 
for i in meta['image file path']:
    name = i.split('/')[0]
    for x in range(1, 10):
        if os.path.exists(f'Train-Mass-Roi/manifest-LyDgOQGl3853937313152078328/CBIS-DDSM/{name}_{x}/'):
            ls =  len(get_all_files_in_folder(f'Train-Mass-Roi/manifest-LyDgOQGl3853937313152078328/CBIS-DDSM/{name}_{x}/'))
            if ls == 1:
                print('++++++++++++++++++++++++++++')
                print(ls)
                print(f'Train-Mass-Roi/manifest-LyDgOQGl3853937313152078328/CBIS-DDSM/{name}_{x}/')
            if ls > 2:
                print('++++++++++++++++++++++++++++')
                print(ls)
                print(f'Train-Mass-Roi/manifest-LyDgOQGl3853937313152078328/CBIS-DDSM/{name}_{x}/')

In [ ]:
# Convert DICOM image to PNG image
def procees_dcm(path):
    
    dicom = pydicom.dcmread(path)
    # try : 
    img = dicom.pixel_array
        
    img = apply_windowing(img, dicom)
    img = (img - img.min()) / (img.max() - img.min())

    if dicom.PhotometricInterpretation == "MONOCHROME1":  
        img = 1 - img
    image = np.uint8(img * 255)
    return image

In [ ]:
# Get image in folder origin and processing image to png
# We will get image and mask 
def get_image(path_name, path_name_mask, path_origin, path_origin_mask):
    df = pd.DataFrame(columns=['Image', 'Crop', 'Mask'])
    if os.path.exists('Train/Meta_data/') ==False: os.makedirs('Train/Meta_data/')
    if os.path.exists('Train/Roi/') ==False: os.makedirs('Train/Roi/')
    if os.path.exists('Train/Image/') ==False: os.makedirs('Train/Image/')
    if os.path.exists('Train/Mask/') ==False: os.makedirs('Train/Mask/')
    paths = get_all_files_in_folder(f'{path_origin}/{path_name}')
    if len(paths) > 1:
        print(f'{path_name} has 2 image in img full')
    sz_img = 0
    for path in paths:
        if os.path.exists(f'Train/Meta_data/{path_name}.png') : 
            sz_img = cv2.imread(f'Train/Meta_data/{path_name}.png').shape
            continue
        if os.path.exists(f'Train/Meta_data/') == False: os.makedirs(f'Train/Meta_data/')
        img = procees_dcm(path)
        sz_img = img.shape
        imageio.imwrite(f'Train/Meta_data/{path_name}.png', img)
        # imageio.imwrite(f'Image/{path_name}.png', img)
    # Processing image mask
    paths_mask = get_all_files_in_folder(f'{path_origin_mask}/{path_name_mask}')
    print(f'{path_origin_mask}/{path_name_mask}')
    print(len(paths_mask))
    print(paths_mask)
    for mask in paths_mask:
        img = procees_dcm(mask)
        path_mask = f'Train/Meta_data/{path_name}/{path_name_mask}_mask.png'
        path_roi = f'Train/Meta_data/{path_name}/{path_name_mask}_roi.png'
        # print(sz_img ,  img)
        # print(sz_img , img.shape)
        if sz_img == img.shape or (sz_img[0] == img.shape[0] and sz_img[1] == img.shape[1] ):
            # print(f'Mask/{path_name_mask}_mask.png')
            imageio.imwrite(path_mask, img)
            imageio.imwrite(f'Train/Mask/{path_name_mask}_mask.png', img)
        else:
            # print(f'Roi/{path_name_mask}_roi.png')
            imageio.imwrite(path_roi, img)
            imageio.imwrite(f'Train/Roi/{path_name_mask}_roi.png', img)
    
    
        

In [ ]:
from tqdm.notebook import tqdm


In [ ]:
for i in tqdm(meta.index):
    path_name = meta['image file path'][i].split('/')[0]
    path_name_mask = meta['ROI mask file path'][i].split('/')[0]
    # print(path_name_mask)
    path_origin = 'Train-Mass/manifest-Egq0PU078220738724328010106/CBIS-DDSM'
    path_origin_mask = 'Train-Mass-Roi/manifest-LyDgOQGl3853937313152078328/CBIS-DDSM'
    get_image(path_name=path_name, path_name_mask=path_name_mask, path_origin=path_origin, path_origin_mask =path_origin_mask)

### Combine alls in to mask

In [ ]:
meta['id_image'] = meta['image file path'].apply(lambda x: x.split('/')[0])

In [ ]:
image = np.unique(meta['id_image'])

In [ ]:
data = []
'''
Make sure all mask is true 
'''
for i in tqdm(image):
    paths = glob.glob(f'Meta_data/{i}/*')
    original_image = cv2.imread(f'Meta_data/{i}/{i}.png')
    blank_image = original_image.copy()
    blank_image.fill(0)
    if len(blank_image.shape) < 3: blank_image =  cv2.cvtColor(blank_image, cv2.COLOR_GRAY2RGB)
    # print(blank_image.shape)
    cnt = 0
    # print(paths)
    for path in paths:
        if 'mask' in path:
            cnt+=1
            img = cv2.imread(path)
            blank_image += img
    # print(cnt)
    if cnt > 0:
        print(f'Meta_data/{i}/{i}.png')
        data.append([f'Meta_data/{i}/{i}.png', f'Meta_data/{i}/{i}_mask.png'])
        imageio.imwrite(f'Meta_data/{i}/{i}_mask.png', blank_image)

In [ ]:
meta_seg = pd.DataFrame(columns=['Image', 'Mask'], data=data)

In [ ]:
meta_seg

In [ ]:
meta_seg.to_csv('meta_seg.csv')

In [ ]:
for path in meta_seg.Mask:
    img = cv2.imread(path)
    name = os.path.basename(path)
    imageio.imwrite(f'E:/Processing Data/Mask_full/{name}', img)